# Final Project: Data Prep and Feature Engineering

Author: Alex Searle

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import swifter
import warnings
%matplotlib inline
sns.set_theme(style='darkgrid')
warnings.filterwarnings(action='once')

## Data Preparation

In [2]:
# Reading in all the necessary data
driver_standings_df = pd.read_csv('Data/driver_standings.csv')
races_df = pd.read_csv('Data/races.csv')
results_df = pd.read_csv('Data/results.csv')
constructor_standings_df = pd.read_csv('Data/constructor_standings.csv')

In [3]:
results_df = results_df.merge(races_df[['year','round', 'raceId']], left_on='raceId', right_on='raceId', how='left', suffixes=['','_drop'])
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         26080 non-null  int64  
 1   raceId           26080 non-null  int64  
 2   driverId         26080 non-null  int64  
 3   constructorId    26080 non-null  int64  
 4   number           26080 non-null  object 
 5   grid             26080 non-null  int64  
 6   position         26080 non-null  object 
 7   positionText     26080 non-null  object 
 8   positionOrder    26080 non-null  int64  
 9   points           26080 non-null  float64
 10  laps             26080 non-null  int64  
 11  time             26080 non-null  object 
 12  milliseconds     26080 non-null  object 
 13  fastestLap       26080 non-null  object 
 14  rank             26080 non-null  object 
 15  fastestLapTime   26080 non-null  object 
 16  fastestLapSpeed  26080 non-null  object 
 17  statusId    

In [4]:
drop_columns = []
for column in results_df.columns:
    if '_extra' in column:
        drop_columns.append(column)
results_df.drop(columns=drop_columns, inplace=True)
results_df.drop(columns=['number','position','positionText', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed'], inplace=True)

In [5]:
results_df.dropna(inplace=True)
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   resultId       26080 non-null  int64  
 1   raceId         26080 non-null  int64  
 2   driverId       26080 non-null  int64  
 3   constructorId  26080 non-null  int64  
 4   grid           26080 non-null  int64  
 5   positionOrder  26080 non-null  int64  
 6   points         26080 non-null  float64
 7   statusId       26080 non-null  int64  
 8   year           26080 non-null  int64  
 9   round          26080 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 2.0 MB


## Feature Engineering

In [6]:
def top3_finishes(row, df):
    variable  = df[(df.year == row.year) & (df['round'] < row['round']) & (df.positionOrder < 4) & (df.driverId == row.driverId)].positionOrder.count() / (row['round'] - 1) * 100
    if variable == np.NAN:
        return 0
    else:
        return variable

In [7]:
results_df['top3_driver_season_percentage'] = results_df.swifter.apply(top3_finishes, axis=1, args=(results_df,))
results_df.info()

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

/tmp/ipykernel_10309/4173861290.py:2: RuntimeWarning: invalid value encountered in scalar divide
  variable  = df[(df.year == row.year) & (df['round'] < row['round']) & (df.positionOrder < 4) & (df.driverId == row.driverId)].positionOrder.count() / (row['round'] - 1) * 100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   resultId                       26080 non-null  int64  
 1   raceId                         26080 non-null  int64  
 2   driverId                       26080 non-null  int64  
 3   constructorId                  26080 non-null  int64  
 4   grid                           26080 non-null  int64  
 5   positionOrder                  26080 non-null  int64  
 6   points                         26080 non-null  float64
 7   statusId                       26080 non-null  int64  
 8   year                           26080 non-null  int64  
 9   round                          26080 non-null  int64  
 10  top3_driver_season_percentage  24362 non-null  float64
dtypes: float64(2), int64(9)
memory usage: 2.2 MB


In [8]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   resultId                       26080 non-null  int64  
 1   raceId                         26080 non-null  int64  
 2   driverId                       26080 non-null  int64  
 3   constructorId                  26080 non-null  int64  
 4   grid                           26080 non-null  int64  
 5   positionOrder                  26080 non-null  int64  
 6   points                         26080 non-null  float64
 7   statusId                       26080 non-null  int64  
 8   year                           26080 non-null  int64  
 9   round                          26080 non-null  int64  
 10  top3_driver_season_percentage  24362 non-null  float64
dtypes: float64(2), int64(9)
memory usage: 2.2 MB


In [9]:
results_df['top_3'] = results_df.positionOrder < 4

In [10]:
def avg_finish_position_season(row, df):
    return df[(df.driverId == row.driverId) & (df.year == df.year) & (df['round'] < row['round'])].positionOrder.mean()

In [11]:
results_df['driver_avg_finish_pos_season'] = results_df.swifter.apply(avg_finish_position_season, axis=1, args=(results_df,))

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

In [12]:
def constructor_top_3(row, df):
    return (df[(df['year'] == row.year) & (df.constructorId == row.constructorId) & (
                df['round'] < row['round'])].top_3.sum()) / ((row['round'] - 1) * 2) * 100

In [13]:
results_df['Constructor_Top3_Percent'] = results_df.swifter.apply(constructor_top_3, axis=1, args=(results_df,))

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

/tmp/ipykernel_10309/319467576.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return (df[(df['year'] == row.year) & (df.constructorId == row.constructorId) & (


In [14]:
results_df.fillna(0, inplace=True)

In [15]:
max_round = pd.DataFrame(results_df.groupby(['year'], as_index=False)['round'].max(), columns=['year', 'round'])
max_round['year'] = max_round['year'] + 1

In [16]:
last_race_stats = results_df.merge(max_round, on=['year'], how='left', suffixes=['', '_max_last_szn'])
last_race_stats

,resultId,raceId,driverId,constructorId,grid,positionOrder,points,statusId,year,round,top3_driver_season_percentage,top_3,driver_avg_finish_pos_season,Constructor_Top3_Percent,round_max_last_szn
0,1,18,1,1,1,1,10.0,1,2008,1,0.0,True,0.000000,0.000000,17.0
1,2,18,2,2,5,2,8.0,1,2008,1,0.0,True,0.000000,0.000000,17.0
2,3,18,3,3,7,3,6.0,1,2008,1,0.0,True,0.000000,0.000000,17.0
3,4,18,4,4,11,4,5.0,1,2008,1,0.0,False,0.000000,0.000000,17.0
4,5,18,5,1,3,5,4.0,1,2008,1,0.0,False,0.000000,0.000000,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26075,26081,1110,817,213,19,16,0.0,1,2023,12,0.0,False,9.784000,0.000000,22.0
26076,26082,1110,858,3,18,17,0.0,1,2023,12,0.0,False,16.363636,0.000000,22.0
26077,26083,1110,807,210,0,18,0.0,1,2023,12,0.0,False,12.252174,0.000000,22.0
26078,26084,1110,832,6,4,19,0.0,130,2023,12,0.0,False,9.737374,9.090909,22.0


In [17]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   resultId                       26080 non-null  int64  
 1   raceId                         26080 non-null  int64  
 2   driverId                       26080 non-null  int64  
 3   constructorId                  26080 non-null  int64  
 4   grid                           26080 non-null  int64  
 5   positionOrder                  26080 non-null  int64  
 6   points                         26080 non-null  float64
 7   statusId                       26080 non-null  int64  
 8   year                           26080 non-null  int64  
 9   round                          26080 non-null  int64  
 10  top3_driver_season_percentage  26080 non-null  float64
 11  top_3                          26080 non-null  bool   
 12  driver_avg_finish_pos_season   26080 non-null 

In [18]:
lag_df = results_df.copy()
lag_df['year'] = lag_df['year'] + 1
last_race_stats = last_race_stats.merge(lag_df[['top3_driver_season_percentage', 'driver_avg_finish_pos_season', 'Constructor_Top3_Percent','year', 'round', 'driverId']], left_on=['year', 'round_max_last_szn', 'driverId'],right_on=['year', 'round', 'driverId'], how='left', suffixes=('', '_lag'))

In [19]:
last_race_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26127 entries, 0 to 26126
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   resultId                           26127 non-null  int64  
 1   raceId                             26127 non-null  int64  
 2   driverId                           26127 non-null  int64  
 3   constructorId                      26127 non-null  int64  
 4   grid                               26127 non-null  int64  
 5   positionOrder                      26127 non-null  int64  
 6   points                             26127 non-null  float64
 7   statusId                           26127 non-null  int64  
 8   year                               26127 non-null  int64  
 9   round                              26127 non-null  int64  
 10  top3_driver_season_percentage      26127 non-null  float64
 11  top_3                              26127 non-null  boo

In [20]:
last_race_stats = last_race_stats.sort_values(['year', 'round'], ascending=True)
last_race_stats

,resultId,raceId,driverId,constructorId,grid,positionOrder,points,statusId,year,round,top3_driver_season_percentage,top_3,driver_avg_finish_pos_season,Constructor_Top3_Percent,round_max_last_szn,top3_driver_season_percentage_lag,driver_avg_finish_pos_season_lag,Constructor_Top3_Percent_lag,round_lag
20070,20025,833,642,51,1,1,9.0,1,1950,1,0.0,True,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
20071,20026,833,786,51,2,2,6.0,1,1950,1,0.0,True,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
20072,20027,833,686,51,4,3,4.0,1,1950,1,0.0,True,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
20073,20028,833,704,154,6,4,3.0,12,1950,1,0.0,False,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
20074,20029,833,627,154,9,5,2.0,12,1950,1,0.0,False,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26122,26081,1110,817,213,19,16,0.0,1,2023,12,0.0,False,9.784000,0.000000,22.0,0.000000,9.918103,2.380952,22.0
26123,26082,1110,858,3,18,17,0.0,1,2023,12,0.0,False,16.363636,0.000000,22.0,NaN,NaN,NaN,NaN
26124,26083,1110,807,210,0,18,0.0,1,2023,12,0.0,False,12.252174,0.000000,22.0,NaN,NaN,NaN,NaN
26125,26084,1110,832,6,4,19,0.0,130,2023,12,0.0,False,9.737374,9.090909,22.0,42.857143,9.936416,45.238095,22.0


In [21]:
results_df = last_race_stats.copy()
results_df.dropna(inplace=True)
results_df.drop(columns=['resultId', 'raceId', 'constructorId'], inplace=True)
results_df[['positionOrder', 'grid']].astype('int')
results_df['year'] = pd.to_datetime(results_df['year'], format='%Y')
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18114 entries, 19881 to 26125
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   driverId                           18114 non-null  int64         
 1   grid                               18114 non-null  int64         
 2   positionOrder                      18114 non-null  int64         
 3   points                             18114 non-null  float64       
 4   statusId                           18114 non-null  int64         
 5   year                               18114 non-null  datetime64[ns]
 6   round                              18114 non-null  int64         
 7   top3_driver_season_percentage      18114 non-null  float64       
 8   top_3                              18114 non-null  bool          
 9   driver_avg_finish_pos_season       18114 non-null  float64       
 10  Constructor_Top3_Percent           

In [22]:
results_df.to_csv('Data/Modeling_v1.csv')

In [23]:
results_df.head(40)

,driverId,grid,positionOrder,points,statusId,year,round,top3_driver_season_percentage,top_3,driver_avg_finish_pos_season,Constructor_Top3_Percent,round_max_last_szn,top3_driver_season_percentage_lag,driver_avg_finish_pos_season_lag,Constructor_Top3_Percent_lag,round_lag
19881,579,1,1,9.0,1,1951-01-01,1,0.000000,True,0.000000,0.0,7.0,50.000000,4.780488,83.333333,7.0
19882,579,1,1,9.0,1,1951-01-01,1,0.000000,True,0.000000,0.0,7.0,50.000000,4.780488,83.333333,7.0
19883,641,6,2,6.0,1,1951-01-01,1,0.000000,True,0.000000,0.0,7.0,0.000000,6.583333,83.333333,7.0
19884,642,2,3,4.0,1,1951-01-01,1,0.000000,True,0.000000,0.0,7.0,33.333333,5.962963,83.333333,7.0
19885,763,4,4,3.0,11,1951-01-01,1,0.000000,False,0.000000,0.0,7.0,0.000000,7.750000,83.333333,7.0
19886,640,5,5,2.0,12,1951-01-01,1,0.000000,False,0.000000,0.0,7.0,0.000000,12.142857,8.333333,7.0
19887,647,7,6,0.0,12,1951-01-01,1,0.000000,False,0.000000,0.0,7.0,16.666667,8.500000,16.666667,7.0
19888,647,7,6,0.0,12,1951-01-01,1,0.000000,False,0.000000,0.0,7.0,16.666667,8.500000,16.666667,7.0
19889,589,19,7,0.0,12,1951-01-01,1,0.000000,False,0.000000,0.0,7.0,16.666667,13.538462,8.333333,7.0
19891,627,8,9,0.0,13,1951-01-01,1,0.000000,False,0.000000,0.0,7.0,33.333333,11.068966,16.666667,7.0
